In [1]:
import pandas as pd

# https://files.grouplens.org/datasets/movielens/ml-25m.zip
movies = pd.read_csv("C:/Users/DELL/Desktop/Untitled Folder/ml-25m/movies.csv")

In [2]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


# CLEANING MOVIE DATA WITH 'REGEX'

In [3]:
import re        ##python regular expression library

def clean_title(title):
    return re.sub("[^a-zA-Z0-9] ","",title) #look for any char that arent a digit,space,lwcase,upcase and remove them 
                                            #removes parenthesis dashes

In [4]:
movies["clean_title"] = movies["title"].apply(clean_title) #new column in datafreame and clean title

In [5]:
movies

,movieId,title,genres,clean_title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story (1995)
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji (1995)
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II (1995)
...,...,...,...,...
62418,209157,We (2018),Drama,We (2018)
62419,209159,Window of the Soul (2001),Documentary,Window of the Soul (2001)
62420,209163,Bad Poems (2018),Comedy|Drama,Bad Poems (2018)
62421,209169,A Girl Thing (2001),(no genres listed),A Girl Thing (2001)


# CREATING A TFIDF MATRIX

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer #turning title into number
vectorizer = TfidfVectorizer(ngram_range =(1,2)) #instead of looking just a words in the title it also look words as "toy story""story 1995"

tfidf = vectorizer.fit_transform(movies["clean_title"])


# CREATING A SEARCH FUNCTION

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(title):
    title = clean_title(title)
    query_vec = vectorizer.transform([title])
    similarity = cosine_similarity(query_vec,tfidf).flatten() # compare the query term to each of the title return similar titles
    indices = np.argpartition(similarity, -5)[-5:] # to get five most similar titles
    results = movies.iloc[indices][::-1] # index imp titles from top
    return results

In [8]:
#results
#similarity
#query_vec

# BUILIDING AND INTERACTIVE SEARCH BOX

In [9]:
from ipywidgets import widgets
from IPython.display import display

movie_input = widgets.Text(
    value = "Toy Story",
    description = "Type a movie",
    disabled = False
)

#movie_input

movie_list = widgets.Output()

def on_type(data):
    with movie_list:
        movie_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            display(search(title))

movie_input.observe(on_type, names = "value")

display(movie_input,movie_list)


Text(value='Toy Story', description='Type a movie')

Output()

# READING IN MOVIE RATED DATA

In [10]:
ratings = pd.read_csv("C:/Users/DELL/Desktop/Untitled Folder/ml-25m/ratings.csv")

In [11]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [12]:
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [13]:
movie_id = 1 

similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] >= 4)]["userId"].unique()

In [14]:
similar_users

array([     3,      5,      8, ..., 162530, 162533, 162534], dtype=int64)

In [15]:
similar_users_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]

In [16]:
similar_users_recs = similar_users_recs.value_counts() / len(similar_users) #count how many times each movie appears

similar_users_recs = similar_users_recs[similar_users_recs > .1] #only movies greater than 10%

In [17]:
similar_users_recs

1       0.499483
318     0.421226
260     0.367817
296     0.353337
356     0.322708
          ...   
1148    0.103609
1527    0.102867
4995    0.102522
778     0.102495
34      0.100162
Name: movieId, Length: 90, dtype: float64

# HOW MUCH USERS LIKE THE MOVIE

In [18]:
all_users = ratings[(ratings["movieId"].isin(similar_users_recs.index)) & (ratings["rating"] > 4)]

In [19]:
all_users

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
29,1,4973,4.5,1147869080
48,1,7361,5.0,1147880055
72,2,110,5.0,1141416589
76,2,260,5.0,1141417172
...,...,...,...,...
25000058,162541,4995,5.0,1240951903
25000062,162541,5618,4.5,1240953299
25000065,162541,5952,5.0,1240952617
25000078,162541,7153,5.0,1240952613


In [20]:
all_users_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())

In [21]:
all_users_recs

318     0.345497
296     0.287399
2571    0.246370
356     0.237518
593     0.228071
          ...   
3114    0.054220
2716    0.053892
34      0.052729
1073    0.049232
1148    0.047922
Name: movieId, Length: 90, dtype: float64

# CREATING A RECOMMENDATION SYSTEM

In [22]:
rec_percentages = pd.concat([similar_users_recs, all_users_recs], axis = 1)
rec_percentages.columns = ["similar","all"]

In [23]:
rec_percentages

,similar,all
1,0.499483,0.125923
32,0.151608,0.101253
34,0.100162,0.052729
47,0.208385,0.145852
50,0.266594,0.202432
...,...,...
33794,0.117160,0.074036
58559,0.180461,0.147871
60069,0.134371,0.077038
68954,0.119070,0.065565


In [24]:
rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]

In [25]:
rec_percentages = rec_percentages.sort_values("score", ascending=False)

In [26]:
rec_percentages

,similar,all,score
1,0.499483,0.125923,3.966586
3114,0.170357,0.054220,3.141967
4886,0.166645,0.071489,2.331060
6377,0.166565,0.072960,2.282977
1073,0.111591,0.049232,2.266621
...,...,...,...
58559,0.180461,0.147871,1.220392
318,0.421226,0.345497,1.219189
4973,0.136148,0.113481,1.199744
2959,0.252380,0.218792,1.153517


In [27]:
rec_percentages.head(10).merge(movies, left_index = True , right_on = "movieId")

,similar,all,score,movieId,title,genres,clean_title
0,0.499483,0.125923,3.966586,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story (1995)
3021,0.170357,0.054220,3.141967,3114,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 2 (1999)
4780,0.166645,0.071489,2.331060,4886,"Monsters, Inc. (2001)",Adventure|Animation|Children|Comedy|Fantasy,MonstersInc(2001)
6258,0.166565,0.072960,2.282977,6377,Finding Nemo (2003),Adventure|Animation|Children|Comedy,Finding Nemo (2003)
1047,0.111591,0.049232,2.266621,1073,Willy Wonka & the Chocolate Factory (1971),Children|Comedy|Fantasy|Musical,Willy Wonka the Chocolate Factory (1971)
8246,0.154207,0.069109,2.231373,8961,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy,IncrediblesThe (2004)
580,0.151449,0.068159,2.221989,588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical,Aladdin (1992)
1120,0.103609,0.047922,2.162033,1148,Wallace & Gromit: The Wrong Trousers (1993),Animation|Children|Comedy|Crime,Wallace GromitThe Wrong Trousers (1993)
359,0.184730,0.086585,2.133522,364,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX,Lion KingThe (1994)
587,0.128060,0.060551,2.114900,595,Beauty and the Beast (1991),Animation|Children|Fantasy|Musical|Romance|IMAX,Beauty and the Beast (1991)


In [33]:
def find_similar_movies(movie_id):
    similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] >= 4)]["userId"].unique()
    similar_users_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
    
    similar_users_recs = similar_users_recs.value_counts() / len(similar_users) #count how many times each movie appears
    similar_users_recs = similar_users_recs[similar_users_recs > .1] #only movies greater than 10%
    
    all_users = ratings[(ratings["movieId"].isin(similar_users_recs.index)) & (ratings["rating"] > 4)]
    all_users_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())
    
    rec_percentages = pd.concat([similar_users_recs, all_users_recs], axis = 1)
    rec_percentages.columns = ["similar","all"]
    
    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    
    rec_percentages = rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(movies, left_index = True , right_on = "movieId")[["score","title","genres"]]
    

In [34]:
movie_name_input = widgets.Text(
    value = "Toy Story",
    description = "Movie title:",
    disabled = False
)
recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) > 5:
            results = search(title)
            movie_id = results.iloc[0]["movieId"]
            display(find_similar_movies(movie_id))
            
movie_name_input.observe(on_type,names = "value")
display(movie_name_input, recommendation_list)
        

Text(value='Toy Story', description='Movie title:')

Output()